In [ ]:
##install and import necessary modules
##this code was originally designed and run in google colab
##use outside of colab may require modification
##if using colab, you may need to restart your runtime after installing modules,
##depending on enviornment at time of code running.

!pip install scikit-learn==1.5.2
!pip install tensorflow==2.12.1
!pip install xgboost==2.0.2
import sys
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, f1_score, roc_curve, auc, precision_recall_curve, recall_score, confusion_matrix, brier_score_loss
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

print("Python version:", sys.version)
print("scikit-learn version:", sklearn.__version__)
print("XGBoost version:", xgb.__version__)

In [ ]:
##import your dataset

##mount google drive if using in colab. Replace <MOUNT_POINT> with the directory where you want to mount the drive (e.g., /content/drive).
drive.mount('<MOUNT_POINT>')

# Replace <YOUR_FILE_PATH> with the actual path inside your Google Drive (e.g., My Drive/FileNameHere).
file_path = '<MOUNT_POINT>/<YOUR_FILE_PATH>.csv'

In [ ]:
##import data and specify missing values
raw_data = pd.read_csv(file_path, na_values=['NA', 'N/A', 'NULL', ' ', '', '-99', '-98', '-99.0', '-99.00', '-98.0', '-98.00', 'NaN'])

##filter data such that we only include non-missing data, and pts transfused more than unitsCutoff (10 for UMT)
unitsCutoff=10.0
data = raw_data[raw_data['UnitsWBandRBC'].notna() & (raw_data['UnitsWBandRBC'] >= unitsCutoff)]

##reset indices of the df
data.reset_index(drop=True, inplace=True)

In [ ]:
##check dataframe to ensure it appears as it should
data.head()

In [ ]:
##check for missing data
data.isnull().sum(axis=0)

In [ ]:
##create a dataframe of all complications and variables not included in this model version.  We can remove all of these from the X data set and pick one to be
#our Y dataset

complications_df=pd.DataFrame()
complications_list= ['HC_CLABSI', 'HC_DEEPSSI', 'HC_DVTHROMBOSIS', 'HC_ALCOHOLWITHDRAWAL', 'HC_CARDARREST', 'HC_CAUTI',
                    'HC_EMBOLISM', 'HC_EXTREMITYCS', 'HC_INTUBATION', 'HC_KIDNEY', 'HC_MI', 'HC_ORGANSPACESSI',
                    'HC_OSTEOMYELITIS', 'HC_RESPIRATORY', 'HC_RETURNOR', 'HC_SEPSIS', 'HC_STROKECVA', 'HC_SUPERFICIALINCISIONSSI',
                    'HC_PRESSUREULCER', 'HC_UNPLANNEDICU', 'HC_VAPNEUMONIA',
                    'MORTALITY', 'Mortality3Hr', 'Mortality6Hr', 'Mortality12Hr', 'Mortality24Hr', 'Mortality14d', 'Mortality30d',
                    'HOSPDISCHARGEDISPOSITION', 'EDDISCHARGEDISPOSITION', 'EDDISCHARGEHRS', 'EDDISCHARGEDAYS', 'FINALDISCHARGEHRS', 'FINALDISCHARGEDAYS',
                    'TOTALICULOS', 'TOTALVENTDAYS', 'FacilityKey',
                    'WITHDRAWALLST', 'WITHDRAWALLSTHRS', 'WITHDRAWALLSTDAYS',
                    'VTEPROPHYLAXISDAYS', 'VTEPROPHYLAXISHRS', 'VTEPROPHYLAXISTYPE',
                    'ESLIVER', 'ESPELVIS', 'ESSPLEEN', 'ESKIDNEY', 'ESRETROPERI', 'ESVASCULAR', 'ESOTHER', 'ES_UK', 'ES_NA', 'ANGIOGRAPHYHRS', 'ANGIOGRAPHYDAYS',
                    'ISS_05', 'AIS_FACE', 'AIS_NECK', 'AIS_HEAD', 'AIS_THORAX', 'AIS_ABDOMEN', 'AIS_SPINE', 'AIS_UPPEREX', 'AIS_LOWEREX', 'AIS_SKIN', 'AIS_OTHER',
                    'ICPEVDRAIN', 'ICPPARENCH', 'ICPO2MONITOR', 'ICPJVBULB', 'ICPNONE', 'ICP_NA', 'ICP_UK',
                    'DRGSCR_AMPHETAMINE', 'DRGSCR_BARBITURATE', 'DRGSCR_BENZODIAZEPINES', 'DRGSCR_COCAINE', 'DRGSCR_METHAMPHETAMINE',
                    'DRGSCR_ECSTASY', 'DRGSCR_METHADONE', 'DRGSCR_OPIOID', 'DRGSCR_OXYCODONE', 'DRGSCR_PHENCYCLIDINE', 'DRGSCR_TRICYCLICDEPRESS',
                    'DRGSCR_CANNABINOID', 'DRGSCR_OTHER', 'DRGSCR_NONE', 'DRGSCR_NOTTESTED', 'DRGSCR_UK', 'DRGSCR_NA', 'ALCOHOLSCREEN', "ALCOHOLSCREENRESULT",
                     'CC_ADHD', 'CC_ADLC', 'CC_ALCOHOLISM', 'CC_ANGINAPECTORIS', 'CC_ANTICOAGULANT', 'CC_BLEEDING', 'CC_CHEMO', 'CC_CIRRHOSIS', 'CC_CONGENITAL',
                     'CC_COPD', 'CC_CVA', 'CC_DEMENTIA', 'CC_DIABETES', 'CC_DISCANCER', 'CC_FUNCTIONAL', 'CC_CHF', 'CC_HYPERTENSION', 'CC_MI', 'CC_PAD',
                     'CC_PREMATURITY', 'CC_MENTALPERSONALITY', 'CC_RENAL', 'CC_SMOKING', 'CC_STEROID', 'CC_SUBSTANCEABUSE', 'mFI'
                     ,'IntracranialVascularInjury', 'BrainStemInjury', 'EDH', 'SAH', 'SDH', 'IPH', 'SkullFx', 'DAI', 'NeckVascularInjury', 'ThoracicVascularInjury'
                     ,'AeroDigestiveInjury', 'CardiacInjury', 'LungInjury', 'AbdominalVascular', 'RibFx', 'KidneyInjury', 'StomachInjury', 'SpleenInjury', 'UroGenInternalInjury'
                     , 'SCI', 'SpineFx', 'UEAmputation', 'UEVascularInjury', 'UELongBoneFx', 'LEVascularInjury', 'PelvicFx', 'LEAmputation', 'PancreasInjury'
                     , 'LELongBoneFx', 'LiverInjury', 'ColorectalInjury', 'SmallBowelInjury', 'NumberOfInjuries'
                     , 'AngioInFour', 'HmgCtrlSurgInFour'
                     , 'missingGCS', 'missingAge', 'missingSex', 'missingType', 'missingSBP', 'missingHR', 'missingRR', 'missingPulseOx', 'missingHeight', 'missingWeight'
                     , 'TotalDeathsFacility', 'FacilityKey', 'TM_NA', 'TM_UK', 'PROTDEV_NA', 'PROTDEV_UK', 'VPO_NA', 'VPO_UK'
                    ]
for c in complications_list:
    complications_df[c] = data[c]
complications_df

In [ ]:
##this is where we choose our outcome variable, in this case, 'Mortality6Hr', and move it to a separate dataframe
Y_data = pd.DataFrame()
Y_data['MORTALITY'] = data['Mortality6Hr']
Y_data

In [ ]:
##clean Y_data by replacing "Yes" and "No" vcalues with 0's and 1's

Y_data['MORTALITY'] = Y_data['MORTALITY'].replace({'Yes': 1, 'No': 0})
Y_data

In [ ]:
##remove all unwanted variables as defined above from the input space
X_data = data.drop(columns=complications_list)
X_data.shape

In [ ]:
##need to remove any cases with missing data for our outcome variable
Missing_Y = Y_data.isnull().sum(axis=0)
Missing_Y

In [ ]:
##here we find which rows in Y have missing values
bad_row_index_list=[]
for index, row in Y_data.iterrows():
    n_missings=row.isnull().sum()
    if n_missings>0:
        bad_row_index_list.append(index)
bad_row_index_list

In [ ]:
##now remove the bad rows in Y
Y_clean = Y_data.drop(bad_row_index_list, axis=0)
Y_clean

In [ ]:
##ensure all cases with missing values for the outcome have been dropped
Missing_Y_clean = Y_clean.isnull().sum(axis=0)
Missing_Y_clean

In [ ]:
##and remove bad rows in X
X_data=X_data.drop(bad_row_index_list, axis=0)

In [ ]:
##check which variables in the input space have missing variables
Missing = X_data.isnull().sum(axis=0)
Missing[Missing>0]

In [ ]:
##order variables with missing data by percentage
data_missing = (X_data.isnull().sum(axis=0)/X_data.shape[0]) * 100
data_missing

In [ ]:
##display variables withOUT mising data
data_missing[data_missing == 0].index

In [ ]:
#remove the good columns (no missing values) from data_missing
data_missing = data_missing.drop(data_missing[data_missing == 0].index)
data_missing

In [ ]:
#sort this in ascending order
data_missing = data_missing.sort_values(ascending=False)
data_missing

In [ ]:
##prepare to drop variables with >50% missing values.
dropCutoff=50
bad_column_names = data_missing[data_missing >=dropCutoff].index
bad_column_names

In [ ]:
##actually drop bad variables
X_data_new=X_data.drop(columns=bad_column_names, axis=1)

##check for which variables still have missing data (<50% missing values)
Missing = X_data_new.isnull().sum(axis=0)
Missing[Missing>0]

In [ ]:
#check for columns with less than 50% missing that need to be cleaned
to_be_cleaned_column_names = data_missing[data_missing <50].index
to_be_cleaned_column_names

In [ ]:
##perform median imputation for continuous variable and mode imputation for categorical
for c in to_be_cleaned_column_names:
    v=X_data_new[c]#get values in this column
    v_valid=v[~v.isnull()] # get valid values
    if X_data_new[c].dtype == np.dtype('O'): # non-numeric values
        X_data_new[c]=X_data_new[c].fillna(v.value_counts().index[0]).astype(object) # the most frequent category
    else: # numeric
        X_data_new[c]=X_data_new[c].fillna(v_valid.median()) #replace nan with median value

In [ ]:
##confirm no more missing data in imput space
X_data_new.isnull().sum().sum()

In [ ]:
##verify cleaned dataframe appears as intended
X_data_new.head()

In [ ]:
##remove any additional variables necessary
## Remove the "RACE" and "TRANSPORTMODE" columns, as these are composite varibles that have already been 1 hot encoded
##primarypaymentmethod unlikely to be available on arrival
##remove any existing transfusion related variables, as that is not available on arrival
##remove any hemorrhage control surgery variables, as not available on arrival
columns_to_remove = ['RACE', 'TRANSPORTMODE', 'UnitsWBandRBC','PRIMARYMETHODPAYMENT', 'PLASMAUNITS', 'RoundedUnitsWBandRBC', 'TransfusionRange',
                    'BLOODBINARY', 'PLASMABINARY', 'PLATELETSBINARY', 'CRYOBINARY', 'WHOLEBLOODBINARY', 'HMRRHGCTRLSURGHRS', 'HMRRHGCTRLSURGDAYS',
                    'Units5to9']
X_data_new = X_data_new.drop(columns=columns_to_remove, errors='ignore')

In [ ]:
##first we will convert No's and Yes's to 0's and 1's to minimize the amount of double variables (want to avoid Yes/Nos being converted to 1-hot variables)
##want code to be reusable between different populations of input data.  Not every population will have all of these variables
##Therefore, will do everything within separate try/except blocks
try:
    X_data_new= X_data_new.replace({True: 1, 'Yes': 1, "Female": 1, False: 0, 'No': 0, "Male": 0})
except:
    pass
try:
    X_data_new['ETHNICITY'] = X_data_new['ETHNICITY'].replace({'Hispanic or Latino': 1, 'Not Hispanic or Latino': 0})
except:
    pass
try:
    X_data_new['EMSGCSEYE'] = X_data_new['EMSGCSEYE'].replace({'None': 1, 'To pressure': 2, 'To sound': 3,
                                                               'Spontaneous': 4})
except:
    pass
try:
    X_data_new['GCSEYE'] = X_data_new['GCSEYE'].replace({'None': 1, 'To pressure': 2, 'To sound': 3, 'Spontaneous': 4})
except:
    pass
try:
    X_data_new['EMSGCSVERBAL'] = X_data_new['EMSGCSVERBAL'].replace({'None': 1, 'Sounds': 2, 'Words': 3,
                                                                     'Confused': 4, 'Oriented': 5})
except:
    pass
try:
    X_data_new['EMSGCSMOTOR'] = X_data_new['EMSGCSMOTOR'].replace({'None': 1, 'Extension': 2, 'Abnormal Flexion': 3,
                                                                 'Normal Flexion': 4, 'Localising': 5, 'Obeys commands': 6})
except:
    pass
try:
    X_data_new['TBIGCSMOTOR'] = X_data_new['TBIGCSMOTOR'].replace({'None': 1, 'Extension': 2, 'Abnormal Flexion': 3,
                                                                 'Normal Flexion': 4, 'Localising': 5, 'Obeys commands': 6})
except:
    pass
try:
    X_data_new['GCSVERBAL'] = X_data_new['GCSVERBAL'].replace({'None': 1, 'Sounds': 2, 'Words': 3,
                                                               'Confused': 4, 'Orientated': 5})
except:
    pass
try:
    X_data_new['GCSMOTOR'] = X_data_new['GCSMOTOR'].replace({'None': 1, 'Extension': 2, 'Abnormal Flexion': 3,
                                                           'Normal Flexion': 4, 'Localising': 5, 'Obeys commands': 6})
except:
    pass
try:
    X_data_new['RESPIRATORYASSISTANCE'] = X_data_new['RESPIRATORYASSISTANCE'].replace({'Assisted Respiratory Rate': 1,
                                                                                   'Unassisted Respiratory Rate': 0})
except:
    pass
try:
    X_data_new['SUPPLEMENTALOXYGEN'] = X_data_new['SUPPLEMENTALOXYGEN'].replace({'Supplemental Oxygen': 1,
                                                                             'No Supplemental Oxygen': 0})
except:
    pass

X_data_new.head()

##male coded as 0
##female coded as 1

##not hispanic coded as 0
##hispanic coded as 1

In [ ]:
##need to convert categorical values to numerical values using one-hot encoding
categorical_column=[]
for c in X_data_new.columns:
    if X_data_new[c].dtype == np.dtype('O', 'category'): # non-numeric values
        categorical_column.append(c)
categorical_column

In [ ]:
##check how many variables we need to one-hot encode
len(categorical_column)

In [ ]:
##verify dataframe shape
X_data_new.shape

In [ ]:
##one-hot encode variables above
X_clean=pd.get_dummies(X_data_new, columns=categorical_column, sparse=False)
X_clean.shape

In [ ]:
##verify cleaned true label dataframe shape
Y_clean.shape

In [ ]:
##verify no missing data in the cleaned input space
X_clean.isnull().sum().sum()

In [ ]:
##drop patient ID's
X_clean.drop(['inc_key'], axis=1, inplace=True)

In [ ]:
##replace boolean values in binary variables to numeric values
X_clean = X_clean.replace({True: 1, False: 0})

In [ ]:
##verify dataframe appears as intended
X_clean.head()

In [ ]:
##split cleaned input space into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X_clean, Y_clean, test_size=0.2, random_state=0)

In [ ]:
##Before converting to Numpy arrays, we generate copies of thed data in tensor format to ensure we have access to
##tensor format data if needed

X_train_tensor=X_train.copy()
Y_train_tensor=Y_train.copy()
X_test_tensor=X_test.copy()
Y_test_tesnor=Y_test.copy()

In [ ]:
##convert sets to Numpy arrays:
X_train=X_train.values
Y_train=Y_train.values.reshape(-1)
X_test=X_test.values
Y_test=Y_test.values.reshape(-1)

In [ ]:
##now we have X_train, Y_train, X_test, Y_test as numpy arrays

scaler=StandardScaler()
#get the parameters of the transform
scaler.fit(X_train)
#normalize the features in the training set
X_train_s = scaler.transform(X_train)
#normalize the features in the test set
X_test_s = scaler.transform(X_test)
#normalize the features in the val set
##X_val_s = scaler.transform(X_val)

##lets also scale the tensor copies we created
X_train_tensor_s = scaler.transform(X_train_tensor)
X_test_tensor_s = scaler.transform(X_test_tensor)

In [ ]:
##further split the training set into a training and validation/calibration set
X_train_s_cal, X_val_s_cal, Y_train_cal, Y_val_cal = train_test_split(X_train_s, Y_train, test_size=0.2, random_state=0)

In [ ]:
##now, initialize XGBoost model using parameters determined from gridsearchCV hyperparameter optimization
model_best_gb = xgb.XGBClassifier(random_state=0, colsample_bytree=1.0, learning_rate=0.05, max_depth=3, n_estimators=500, subsample=0.6)

In [ ]:
# Get predicted probabilities for the positive class (mortality)
model_best_gb.fit(X_train_s_cal, Y_train_cal)
y_prob_gbo_mtp = model_best_gb.predict_proba(X_test_s)[:, 1]

# Compute AUROC
auroc_gbo = roc_auc_score(Y_test, y_prob_gbo_mtp)
print(f"AUROC on the test set: {auroc_gbo}")

In [ ]:
# Calibrate the model on the validation set
calibrated_model = CalibratedClassifierCV(estimator=model_best_gb, method='isotonic', cv='prefit')
calibrated_model.fit(X_val_s_cal, Y_val_cal)

In [ ]:
##now, initialize penalized regression model with parameters based on GridsearchCV optimization and fit
model_best_lr=LogisticRegression(C=0.1, max_iter=100, penalty='l1', solver='saga')
model_best_lr.fit(X_train_s_cal, Y_train_cal)

In [ ]:
##test model
y_pred_prob_lro = model_best_lr.predict_proba(X_test_s)[:, 1]
auroc_lro = roc_auc_score(Y_test, y_pred_prob_lro)
print(f'AUROC: {auroc_lro}')

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

def paired_bootstrap_auc_test(
    y_true,
    predA,
    predB,
    n_boot=1000,
    alpha=0.05,
    random_state=None
):
    """
    Compare two correlated AUCs (predA vs. predB) via paired bootstrapping,
    also returning 95% CI for each AUC individually.

    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        Ground truth binary labels (0 or 1).
    predA : array-like of shape (n_samples,)
        Scores (e.g., probabilities) from model/variable A.
    predB : array-like of shape (n_samples,)
        Scores from model/variable B.
    n_boot : int, default=1000
        Number of bootstrap iterations.
    alpha : float, default=0.05
        Significance level (for the (1 - alpha) CIs).
    random_state : int or None
        Random seed for reproducibility.

    Returns
    -------
    results : dict
        Dictionary containing:
        - aucA, aucB: AUC on the original dataset for each method.
        - aucA_ci_lower, aucA_ci_upper: (1 - alpha) CI for A's AUC.
        - aucB_ci_lower, aucB_ci_upper: (1 - alpha) CI for B's AUC.
        - baseline_diff: AUC(A) - AUC(B) on the full dataset (no resampling).
        - mean_diff: Mean difference of AUCs across bootstrap samples.
        - diff_ci_lower, diff_ci_upper: (1 - alpha) CI for AUC difference.
        - p_value: Approx. two-sided p-value for difference in AUC.
    """

    # Convert to NumPy arrays
    y_true = np.asarray(y_true)
    predA = np.asarray(predA)
    predB = np.asarray(predB)

    # Basic checks
    assert len(y_true) == len(predA) == len(predB), "Arrays must all have the same length."
    n = len(y_true)

    # Compute AUC on the full (original) dataset
    aucA = roc_auc_score(y_true, predA)
    aucB = roc_auc_score(y_true, predB)
    baseline_diff = aucA - aucB

    # Random generator
    rng = np.random.default_rng(random_state)

    # Arrays to store bootstrapped AUCs
    aucAs = np.zeros(n_boot)
    aucBs = np.zeros(n_boot)
    diffs = np.zeros(n_boot)

    for i in range(n_boot):
        # 1) Sample n indices with replacement
        sample_idx = rng.integers(0, n, size=n)

        # 2) Create bootstrap samples
        y_samp = y_true[sample_idx]
        A_samp = predA[sample_idx]
        B_samp = predB[sample_idx]

        # 3) Compute AUC for each method
        aucA_samp = roc_auc_score(y_samp, A_samp)
        aucB_samp = roc_auc_score(y_samp, B_samp)

        # 4) Store AUCs and difference
        aucAs[i] = aucA_samp
        aucBs[i] = aucB_samp
        diffs[i] = aucA_samp - aucB_samp

    # Compute individual AUC CIs
    aucA_ci_lower = np.percentile(aucAs, 100 * (alpha / 2))
    aucA_ci_upper = np.percentile(aucAs, 100 * (1 - alpha / 2))

    aucB_ci_lower = np.percentile(aucBs, 100 * (alpha / 2))
    aucB_ci_upper = np.percentile(aucBs, 100 * (1 - alpha / 2))

    # Compute difference CI
    diff_ci_lower = np.percentile(diffs, 100 * (alpha / 2))
    diff_ci_upper = np.percentile(diffs, 100 * (1 - alpha / 2))

    # Approx. two-sided p-value by sign test on "diffs"
    n_neg = np.sum(diffs < 0)
    n_pos = np.sum(diffs > 0)
    p_val = 2.0 * min(n_neg, n_pos) / n_boot
    p_val = min(p_val, 1.0)

    coverage = (1 - alpha) * 100.0  # e.g., 95.0 if alpha=0.05

    return {
        "aucA": aucA,
        "aucB": aucB,
        "aucA_ci_lower": aucA_ci_lower,
        "aucA_ci_upper": aucA_ci_upper,
        "aucB_ci_lower": aucB_ci_lower,
        "aucB_ci_upper": aucB_ci_upper,
        "baseline_diff": baseline_diff,
        "mean_diff": np.mean(diffs),
        "diff_ci_lower": diff_ci_lower,
        "diff_ci_upper": diff_ci_upper,
        "p_value": p_val,
        "coverage": coverage
    }


In [ ]:
# Dictionary to hold the predicted probabilities from models to compare against the main ML model
pairs = {
    "LASSO": y_pred_prob_lro,
}


# Iterate over each model in the pairs dictionary to perform paired bootstrap AUC tests
for var_name, var_array in pairs.items():
    # Perform paired bootstrap AUC comparison between ML model (predA) and comparison model (predB)
    results = paired_bootstrap_auc_test(
        y_true=Y_test,               # True class labels
        predA=y_prob_gbo_mtp,        # XBG model predicted probabilities
        predB=var_array,             # Comparison model (LASSO) predicted probabilities
        n_boot=2000,                 # Number of bootstrap samples for robust estimation
        alpha=0.05,                  # Significance level for 95% confidence interval
        random_state=42              # Seed for reproducibility
    )

    # Format confidence interval output
    coverage_str = f"{results['coverage']:.1f}%"

    # Print results with clear and formatted output
    print(f"--- ML Model vs. {var_name} ---")
    print(f"AUC(ML) = {results['aucA']:.3f}, {coverage_str} CI: "
          f"[{results['aucA_ci_lower']:.3f}, {results['aucA_ci_upper']:.3f}]")
    print(f"AUC({var_name}) = {results['aucB']:.3f}, {coverage_str} CI: "
          f"[{results['aucB_ci_lower']:.3f}, {results['aucB_ci_upper']:.3f}]")
    print(f"AUC diff (ML - {var_name}) = {results['baseline_diff']:.4f}, {coverage_str} CI: "
          f"[{results['diff_ci_lower']:.4f}, {results['diff_ci_upper']:.4f}]")
    print(f"p-value = {results['p_value']:.4f}\n")